<a href="https://colab.research.google.com/github/sliscak/e2n-translator/blob/main/english2narsese.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

INSTALL PREREQ

In [2]:
%%capture
from google.colab import drive
drive.mount('/content/drive')
#!cp "/content/drive/My Drive/research/datasets/dataset.jsonl" "/content"
!git clone https://github.com/huggingface/transformers.git
!pip install ./transformers
!pip install -U nlp
!pip install pytorch-lightning
!pip install tqdm

!rm e2n-translator -rf
!git clone https://github.com/sliscak/e2n-translator.git
!python e2n-translator/dataset/build.py

IMPORT PACKAGES

In [3]:
from transformers import T5Tokenizer, T5Model, T5ForConditionalGeneration, Trainer, TrainingArguments, T5Config, AdamW #T5WithLMHeadModel
import torch
from torch.utils.data import Dataset, DataLoader
import json
import nlp
from pytorch_lightning.core.lightning import LightningModule
import pytorch_lightning as pl
from os import path
from tqdm import tqdm

In [4]:
tokenizer = T5Tokenizer.from_pretrained('t5-base')
PATH = "/content/drive/My Drive/research/models/t5-py"
# model = T5ForConditionalGeneration.from_pretrained('t5-base')

In [5]:
 class PythonDataset(Dataset):
    """Python dataset."""
    """Either preprocess or process data on the fly"""

    def __init__(self, file_path="/content/dataset.txt"):
      self.data = []
      with open(file_path, 'r') as fd:
        pair = []
        for line in tqdm(fd.readlines(), desc='Dataset Loading'):
          if len(line) > 3:
              pair.append(line)
              if len(pair) >= 2:
                self.data.append(pair)
                pair = [] 
        # else:
        #   for line in tqdm(fd.readlines(), desc='Dataset Loading'):
        #     # TODO: remove this part
        #     if len(line) > 3:
        #       self.data.append(line)

    def process(self, pair):
        with torch.no_grad():
          text, narsese = pair
          pattern = f'translate TEXT to NARSESE: "{text}"'

          input_ids = tokenizer.encode(pattern, padding=True, truncation=True, return_tensors='pt')
          labels = tokenizer.encode(narsese, padding=True, truncation=True, return_tensors='pt')
          sample = {
              "input_ids": input_ids,
              "labels": labels,
          }
          return sample

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
        # print(self.data[idx])
        # processing on the fly
        return self.process(self.data[idx])

dataset = PythonDataset()
dataset[0]

Dataset Loading: 100%|██████████| 62/62 [00:00<00:00, 309211.47it/s]


{'input_ids': tensor([[13959,     3,  3463,     4,   382,    12,   445, 25210,  3205,   427,
             10,    96,   667,   210,    40,    19,     3,     9,   686,    13,
           2586,     5,    96,     1]]),
 'labels': tensor([[    3,     2,  2381,    40,     3,    18, 13114,  2586,  3155,     5,
              1]])}

In [6]:
class Pavian(LightningModule):
  def __init__(self): # reset=False
    super().__init__()
    self.model = T5ForConditionalGeneration.from_pretrained('t5-base')
    self.model.train()
    # self.save_hyperparameters()

  def forward(self, kwargs):
    return self.model(**kwargs)

  def configure_optimizers(self):
        optimizer = AdamW(model.parameters(), lr=1e-5)
        return optimizer

  def training_step(self, batch, batch_idx):
        for key in batch.keys():
          batch[key] = batch[key][0]
        outputs = self(batch)
        loss = outputs[0]
        tensorboard_logs = {'train_loss': loss}
        return {'loss': loss, 'log': tensorboard_logs}


  # def validation_step(self, batch, batch_idx):
  #       x, y = batch
  #       y_hat = self(x)
  #       val_loss = F.cross_entropy(y_hat, y)
  #       return val_loss
model = Pavian()

In [28]:
config = {
    # 'gpus': 1, # uncomment to use GPU
    'default_root_dir': '/content/drive/My Drive/research/checkpoints',
    'checkpoint_callback': False,
    'max_epochs': 10,
    # 'precision':16,
}



train_loader = DataLoader(PythonDataset(), batch_size=1, shuffle=True)

trainer = pl.Trainer(**config)
trainer.fit(model, train_loader)

Dataset Loading: 100%|██████████| 62/62 [00:00<00:00, 336848.25it/s]
GPU available: False, used: False
TPU available: False, using: 0 TPU cores

  | Name  | Type                       | Params
-----------------------------------------------------
0 | model | T5ForConditionalGeneration | 222 M 


/usr/local/lib/python3.6/dist-packages/pytorch_lightning/utilities/distributed.py:45: UserWarning: The {log:dict keyword} was deprecated in 0.9.1 and will be removed in 1.0.0
Please use self.log(...) inside the lightningModule instead.

# log on a step or aggregate epoch metric to the logger and/or progress bar
# (inside LightningModule)
self.log('train_loss', loss, on_step=True, on_epoch=True, prog_bar=True)
  warnings.warn(*args, **kwargs)


/usr/local/lib/python3.6/dist-packages/pytorch_lightning/utilities/distributed.py:45: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  warnings.warn(*args, **kwargs)


1

In [8]:
# PATH = "/content/drive/My Drive/research/models/t5-py"
# torch.save(model.state_dict(), PATH)

In [33]:
with torch.no_grad():
    text = '<cat --> animal>.'
    pattern = f'translate TEXT to NARSESE: "Cat is a type of animal."'
    input_ids2 = tokenizer.encode(pattern, return_tensors='pt')
    outputs2 = model.model.generate(input_ids2)
    dec = tokenizer.decode(outputs2[0])
    print(f'Input:\n\t{pattern}\nOutput:\n\t{dec}\n')
    print(f'Ground Truth:\n\t{text}')


Input:
	translate TEXT to NARSESE: "Cat is a type of animal."
Output:
	 ⁇ cat --> animal>.

Ground Truth:
	<cat --> animal>.
